# MapD Runtime UDF support

The *Remote Backend Compiler* (RBC) package implements the MapD client support for defining so-called Runtime UDFs. That is, while MapD server is running, one can register new SQL functions to MapD Calcite server as well as provide their implementations in LLVM IR string form. The RBC package supports creating Runtime UDFs from Python functions.

First, we need to connect RBC to MapD server:

In [1]:
from rbc.mapd import RemoteMapD
mapd = RemoteMapD(user='mapd', password='HyperInteractive', host='127.0.0.1', port=6274, dbname='mapd')

One can use the Runtime UDF functions in SQL queries from several MapD clients, for instance, from `pymapd`, `ibis.mapd`, or `rbc.mapd`. In this demo, we'll use `ibis.mapd`. For that, we connect `ibis.mapd` to MapD server as well:

In [2]:
import ibis
ibis_con = ibis.mapd.connect(user=mapd.user, password=mapd.password,
                             host=mapd.host, port=mapd.port, database=mapd.dbname)

## Create and fill test table

In [3]:
mapd.sql_execute('drop table if exists mytable')
mapd.sql_execute('create table if not exists mytable (x DOUBLE, i INT);');
for _i in range(5):
    mapd.sql_execute('insert into mytable values '+str((_i,)*2));
ibis_con.sql('select x, i from mytable').execute()

,x,i
0,0.0,0
1,1.0,1
2,2.0,2
3,3.0,3
4,4.0,4


## Defining Runtime UDFs from Python

To define a Runtime UDF from a Python function, the function must be decorated with `mapd` decorator:

In [4]:
@mapd('f32(f32)', 'f64(f64)')
def incr(v):
    """Increment float value by one"""
    return v + 1

One can overload existing UDFs (now using Python annotations):

In [5]:
@mapd
def incr(v: 'int32') -> 'int32':
    """Increment integer value by ten"""
    return v + 10

Finally, we register the defined UDFs to MapD server:

In [6]:
mapd.register()

## Using UDFs in a SQL query

In [7]:
t = ibis_con.sql('select x, incr(x), i, incr(i) from mytable')
t[t.i < 3].execute()

,x,EXPR$1,i,EXPR$3
0,0.0,1.0,0,10
1,1.0,2.0,1,11
2,2.0,3.0,2,12


# Advanced: defining UDFs from a LLVM IR string

One can implement UDF registration support to any MapD client that 
is able to provide the UDF implementations in LLVM IR form.

To demonstrate that, let's define such a UDF from a LLVM IR string:

In [8]:
foo_ir = '''
; foo(i, j) -> i * j + 55
define i32 @foo(i32 %.1, i32 %.2) {
entry:
  %.18.i = mul i32 %.2, %.1
  %.33.i = add i32 %.18.i, 55
  ret i32 %.33.i
}
'''
foo_signature = "foo 'int32(int32, int32)'"

and register it to MapD server using its Thrift end-point:

In [9]:
mapd.thrift_call('register_runtime_udf',
                 mapd.session_id,
                 foo_signature,
                 dict(cpu = foo_ir))

Test it:

In [10]:
ibis_con.sql('select i, foo(i, 5) from mytable').execute()

,i,EXPR$1
0,0,55
1,1,60
2,2,65
3,3,70
4,4,75
